In [ ]:
from keras.applications import VGG16
import keras
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import GlobalAveragePooling2D, GlobalMaxPool2D, Flatten, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,KFold
from keras.optimizers import SGD,RMSprop,adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.utils import to_categorical
%matplotlib inline

In [ ]:
## Define path ##
PATH = os.getcwd()
data_path = PATH + '/CT'
data_dir_list = os.listdir(data_path)

IMG_WIDTH=224
IMG_HEIGHT=224

IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)
num_channel=1
num_classes = 2
labels_name={'Covid':0,'non-Covid':1}

img_data_list=[]
labels_list = []

## Retrieval of images and labels
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    for img in img_list:
        input_img = cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize = cv2.resize(input_img,(224,224))
        
        input_img_resize_gray =cv2.resize(input_img_gray,(224,224))
        _, input_img_resize_gray = cv2.threshold(input_img_resize_gray, 128, 255, cv2.THRESH_BINARY)

        moments = cv2.moments(input_img_resize_gray)
        huMoments = cv2.HuMoments(moments)   
        moment_list.append(huMoments)
              
        img_data_list.append(input_img_resize)
        labels_list.append(label)  

In [ ]:
## Image normalization
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
moment_data = np.array(moment_list)
moment_data = moment_data.astype('float32')
moment_data /= 255


In [ ]:
# convert class labels to on-hot encoding
labels = np.array(labels_list)
print(np.unique(labels,return_counts=True))
Y = np_utils.to_categorical(labels, num_classes)


In [ ]:
## Color channel expansion for gray images ##
if num_channel==1:
    if K.common.image_dim_ordering()=='th':
        moment_data= np.expand_dims(moment_data, axis=1) 
    else:
        moment_data= np.expand_dims(moment_data, axis=3) 
else:
    if K.image_dim_ordering()=='th':
        img_data=np.rollaxis(img_data,3,1)
        print (img_data.shape)

In [ ]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
x2,y2 = shuffle(moment_data,Y, random_state=2)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.1, random_state=2)


In [ ]:
### VGG16 initialization ###

# include top should be False to remove the softmax layer
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

In [ ]:
## Feature extraction on train and val set

vgg_features_train = pretrained_model.predict(X_train)
vgg_features_val = pretrained_model.predict(X_test)

vgg_features_train = vgg_features_train.reshape(2052,25088,1)
vgg_features_val = vgg_features_val.reshape(228,25088,1)


In [ ]:
# Reshape moment invariant 1 (training) values into 1 dimension
X_train2_1 = X_train2
Moment_Inv = X_train2_1.reshape(2052,7,1)
VGG_MI = K.concatenate([vgg_features_train, Moment_Inv],axis=1)

# Reshape moment invariant 1 (testing) values into 1 dimension
X_test2_1 = X_test2
Moment_Inv_val = X_test2_1.reshape(228,7,1)
VGG_MI_val = K.concatenate([vgg_features_val, Moment_Inv_val],axis=1)



In [ ]:
import tensorflow as tf
proto_tensor = tf.make_tensor_proto(VGG_MI)
input_array = tf.make_ndarray(proto_tensor)
input_array_shape = input_array[0].shape

proto_tensor_val = tf.make_tensor_proto(VGG_MI_val)
input_array_val = tf.make_ndarray(proto_tensor_val)
print(input_array_val.shape)

In [ ]:
# New Fully Connected layer
model_fusion = Sequential()

#model_fusion.build(input_array_shape)
model_fusion.add(Flatten(input_shape=(25095,1)))
model_fusion.add(Dense(256, activation='relu'))
model_fusion.add(Dropout(0.5))
model_fusion.add(BatchNormalization())
model_fusion.add(Dense(2, activation='sigmoid'))

# sgd = SGD(lr=0.01, decay= 1e-8, momentum= 0.4, nesterov=True)
# model_fusion.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["accuracy"])

# sgd = SGD(lr=0.00004, decay= 1e-7, momentum= 0.4, nesterov=True)
# model_fusion.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["accuracy"])
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model/VGG_Moment_Classifier2.h5', verbose=1, save_best_only=True)

model_fusion.compile(loss='binary_crossentropy', optimizer=adam(lr=0.00005, decay= 1e-5), metrics=['accuracy'])
model_fusion.summary()
Fusion = model_fusion.fit(input_array, y_train, batch_size = 224 , epochs=100, verbose=1, validation_data=(input_array_val, y_test),callbacks=[anne, checkpoint])

In [ ]:
## Saving training output to CSV file
hist_df = pd.DataFrame(Fusion.history) 

hist_csv_file = 'history/VGG_MI_history2.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
history = pd.read_csv("history/VGG_MI_history2.csv")
val_loss = history['val_loss'].to_numpy()
val_acc = history['val_accuracy'].to_numpy()
loss = history['loss'].to_numpy()
acc = history['accuracy'].to_numpy()

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(acc)
plt.plot(loss)
plt.plot(val_acc)
plt.plot(val_loss)
plt.title('Training Accuracy vs Epochs (VGG16 + MI)', fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('Accuracy (%)', fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel('Epoch',fontsize=20)
plt.legend(['Train_acc', 'Train_loss', 'Val_acc', 'Val_loss'], loc='best', fontsize=22)
plt.gcf().set_size_inches(12,10) 
#plt.savefig('C:/Users/CooL/Desktop/Result/All Training vs Loss/VGG_MI_train_graph.jpg')
plt.show()

In [ ]:
%%time
#### Testing phase VGG16+ MI ####
PATH = os.getcwd()
# Define data path
data_path = PATH + '/Testing_Set'
data_dir_list = os.listdir(data_path)

import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix

test_list = []
moment_test_list = []
moment_test_list2 = []
y_pred = []
labels_name ={'Covid-19':0,'Non-Covid-19':1}


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(224,224))
        input_img_moment =cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)     
        moments = cv2.moments(input_img_moment)
        huMoments = cv2.HuMoments(moments)  
        moment_test_list.append(huMoments)       
        test_list.append(input_img_resize)
        y_pred.append(label)

test_image = np.array(test_list)
test_image = test_image.astype('float32')
test_image /= 255
moment_data = np.array(moment_test_list)
moment_data = moment_data.astype('float32')
moment_data /= 255


VGG_features_test = pretrained_model.predict(test_image)
VGG_features_test = VGG_features_test.reshape(200,25088,1)
#predict = np.argmax(Resnet_features_test,axis=1)


Moment_Inv_test = moment_data.reshape(200,7,1)
VGG_MI_test = K.concatenate([VGG_features_test, Moment_Inv_test],axis=1)


proto_tensor = tf.make_tensor_proto(VGG_MI_test)
input_array_test = tf.make_ndarray(proto_tensor)
input_array_shape = input_array[0].shape

predicted_test = model2.predict(input_array_test)
predict = np.argmax(predicted_test,axis=1)

# # predict = overall.predict(input_array)
# # predict = np.argmax(predict,axis=1)

cf_matrix = confusion_matrix(y_pred, predict)
tp = cf_matrix[0][0]
fn = cf_matrix[0][1]
fp = cf_matrix[1][0]
tn = cf_matrix[1][1]

TrueP = '{0:.2%}'.format(tp/(tp+fn))
FalseN = '{0:.2%}'.format(fn/(tp+fn))
FalseP = '{0:.2%}'.format(fp/(fp+tn))
TrueN = '{0:.2%}'.format(tn/(fp+tn))
print(TrueP, FalseN, FalseP, TrueN)

xy_Labels = ['Covid', 'non-Covid']
group_names = ['True Positive', 'False Negative', 'False Positive','True Negative']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
group_percentage = [TrueP, FalseN, FalseP, TrueN]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentage)]
labels = np.asarray(labels).reshape(2,2)


## Confusion matrix VGG16 + MI ##
plt.title('(Moment Invariant + VGG-16)', fontsize = 18)
sns.heatmap(cf_matrix, fmt='', annot=labels, annot_kws={'size':18} ,xticklabels=xy_Labels, yticklabels=xy_Labels, cmap='PuBuGn')
plt.xlabel('Predicted label', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('True label', fontsize = 15) # y-axis label with fontsize 15
#plt.savefig('C:/Users/CooL/Desktop/Result/VGG_Moment Invariant/VGG_Mi_CF.jpg')
plt.show()


from sklearn.metrics import auc, roc_curve
import pandas as pd

predict_list = []
predict_1 = model2.predict(input_array_test)

fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_pred, predict_1[:,1])

auc_keras = auc(fpr_keras, tpr_keras)
optimum = thresholds_keras[np.argmax(tpr_keras - fpr_keras)]
print('Optimal cut off point = ',optimum)
print('AUC = ', auc_keras)

gmeans = np.sqrt(tpr_keras * (1-fpr_keras))
ix = np.argmax(gmeans)
optimal = thresholds_keras[ix]
print('Optimal cut off point = ',thresholds_keras[ix])

x = optimal # Best => x = 0.98892

# for j in range(0,21):
#     for i in range(0, len(predict_1)):
#         if predict_1[i][1] >= x:
#             predict_list.append(1)
#         else:
#             predict_list.append(0)
#     print(confusion_matrix(y_pred, predict_list), 'x =',"%.5f" % x, '\n')
#     predict_list.clear()
#     x+=0.01



### ROC Curves ###
plt.figure(figsize=(6,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.scatter(fpr_keras[ix],tpr_keras[ix], marker='o', color='black', label='Optimal Threshold = {:.4f})'.format(optimal))
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.title('ROC (VGG-16 + Moment Invariant)',fontsize=22)
plt.legend(loc='best', fontsize=15)
#plt.savefig('C:/Users/CooL/Desktop/Result/VGG_Moment Invariant/VGG_MIgraph.jpg')
plt.show()

